In [50]:
import pandas as pd
import matplotlib.pyplot as plt
import random
import math

In [68]:
games_data = pd.read_csv('nba_games_stats.csv')
games_data = games_data[games_data.Home == 'Home'] # remove duplicate games
games_data = games_data[games_data.Date < '2015-10-27'] # remove duplicate games
games_data.shape

(4920, 41)

In [52]:
elo_data = pd.read_csv('nbaallelo.csv')
elo_data.columns

Index(['gameorder', 'game_id', 'lg_id', '_iscopy', 'year_id', 'date_game',
       'seasongame', 'is_playoffs', 'team_id', 'fran_id', 'pts', 'elo_i',
       'elo_n', 'win_equiv', 'opp_id', 'opp_fran', 'opp_pts', 'opp_elo_i',
       'opp_elo_n', 'game_location', 'game_result', 'forecast', 'notes'],
      dtype='object')

In [85]:
initial_elos = {}
for i, first_game in elo_data[elo_data.year_id == 2015][elo_data.seasongame == 1].iterrows():
    initial_elos[first_game.team_id] = first_game.elo_i
initial_elos
#sorted(initial_elos.items())

{'HOU': 1596.4641,
 'LAL': 1421.9985,
 'NOP': 1457.2185,
 'ORL': 1359.4265,
 'SAS': 1699.5024,
 'DAL': 1592.0101,
 'BOS': 1378.5464,
 'BRK': 1518.2666,
 'MIL': 1317.8491,
 'CHO': 1511.0031,
 'DEN': 1473.0493,
 'DET': 1389.2994,
 'IND': 1532.6748,
 'PHI': 1316.4243,
 'MEM': 1578.4064,
 'MIN': 1513.3354,
 'WAS': 1541.4388,
 'MIA': 1579.3748,
 'NYK': 1534.4419,
 'CHI': 1547.2185,
 'PHO': 1560.1208,
 'OKC': 1619.7805,
 'POR': 1567.8318,
 'GSW': 1590.7976,
 'SAC': 1431.1567,
 'TOR': 1550.7469,
 'ATL': 1490.1147,
 'UTA': 1373.317,
 'CLE': 1463.7213,
 'LAC': 1630.6335}

In [84]:
def elo():
    every_elo = {'HOU': 1596.4641,
 'LAL': 1421.9985,
 'NOP': 1457.2185,
 'ORL': 1359.4265,
 'SAS': 1699.5024,
 'DAL': 1592.0101,
 'BOS': 1378.5464,
 'BRK': 1518.2666,
 'MIL': 1317.8491,
 'CHO': 1511.0031,
 'DEN': 1473.0493,
 'DET': 1389.2994,
 'IND': 1532.6748,
 'PHI': 1316.4243,
 'MEM': 1578.4064,
 'MIN': 1513.3354,
 'WAS': 1541.4388,
 'MIA': 1579.3748,
 'NYK': 1534.4419,
 'CHI': 1547.2185,
 'PHO': 1560.1208,
 'OKC': 1619.7805,
 'POR': 1567.8318,
 'GSW': 1590.7976,
 'SAC': 1431.1567,
 'TOR': 1550.7469,
 'ATL': 1490.1147,
 'UTA': 1373.317,
 'CLE': 1463.7213,
 'LAC': 1630.6335}
    upsets = 0
    tied_elo = 0
    for index, game in games_data.iterrows():
        home_team = game.Team
        home_team_rating = every_elo[home_team] + 100

        opponent = game.Opponent
        opponent_rating = every_elo[opponent]

        print("{} (rating: {}) vs {} (rating: {})".format(home_team, home_team_rating, opponent, opponent_rating))

        if home_team_rating > opponent_rating:
            print("Predicted Score Margin: {} by {}".format(home_team, ((home_team_rating - opponent_rating)/28) ))
        elif home_team_rating < opponent_rating:
            print("Predicted Score Margin: {} by {}".format(opponent, ((opponent_rating - home_team_rating)/28) ))
        else:
            tied_elo += 1
            print("Predicted Score: ELO's are tied! Could go either way")

        if game.WINorLOSS == "W":
            S = 1
            winner_loser_elo_diff = home_team_rating - opponent_rating
            if home_team_rating < opponent_rating:
                upsets += 1
        else:
            S = 0
            winner_loser_elo_diff = opponent_rating - home_team_rating
            if home_team_rating > opponent_rating:
                upsets += 1

        expected_home_win = ( 1 / (1 + 10 ** ((opponent_rating - home_team_rating) / 400)) )
        expected_away_win = 1 - expected_home_win

        print(expected_home_win)
        print(expected_away_win)
        print(expected_home_win + expected_away_win)

        margin_of_victory = abs(game.TeamPoints - game.OpponentPoints)
        K = 20 * ( pow((margin_of_victory + 3), 0.8) / (7.5 * 0.006 * winner_loser_elo_diff) )

        home_team_rating += K * (S - expected_home_win)
        opponent_rating += K * (abs(S - 1) - expected_away_win)
        
        every_elo[home_team] = home_team_rating
        every_elo[opponent] = opponent_rating

        print("Score: {} {}, {} {}".format(home_team, game.TeamPoints, opponent, game.OpponentPoints))
        print(K * (S - expected_home_win))
    #print(every_elo)
    print("Upsets: ", upsets)
    print("Tied Elo: ", tied_elo)
    return every_elo
elo()

ing: 80601.84549914717)
Predicted Score Margin: BOS by 11.553496788712177
0.13444640374490535
0.8655535962550946
1.0
Score: TOR 111, BOS 91
-14.609058782015452
TOR (rating: 80363.7385302812) vs NYK (rating: 86499.78780799029)
Predicted Score Margin: NYK by 219.1446170610387
4.569585479466128e-16
0.9999999999999996
1.0
Score: TOR 113, NYK 88
-1.0414899133867699
TOR (rating: 80462.69704036781) vs MIA (rating: 84140.99554487527)
Predicted Score Margin: MIA by 131.3678037324093
6.371676801894371e-10
0.9999999993628323
1.0
Score: TOR 115, MIA 112
-0.5066307098023377
TOR (rating: 80562.19040965801) vs MIL (rating: 83350.67292054169)
Predicted Score Margin: MIL by 99.58866110298861
1.0685471157321794e-07
0.9999998931452885
1.0
Score: TOR 119, MIL 122
-7.141085511509112e-08
TOR (rating: 80662.1904095866) vs DET (rating: 77992.5927583072)
Predicted Score Margin: TOR by 95.34277325997859
0.9999997881610672
2.1183893283982513e-07
1.0
Score: TOR 123, DET 94
5.642836059438607e-07
TOR (rating: 80762

{'HOU': 83822.92919269098,
 'LAL': 82776.70358611083,
 'NOP': 91302.98982226395,
 'ORL': 89203.46630055434,
 'SAS': 81928.06939649003,
 'DAL': 83474.38190850383,
 'BOS': 80608.48366855942,
 'BRK': 84662.08662051398,
 'MIL': 83351.83910483698,
 'CHO': 84661.71139839063,
 'DEN': 84468.38049896466,
 'DET': 77992.31291653393,
 'IND': 84464.60165996396,
 'PHI': 84209.68317048682,
 'MEM': 83224.07199638312,
 'MIN': 83410.83132781081,
 'WAS': 82355.75435941677,
 'MIA': 84142.31499687686,
 'NYK': 86501.62820535082,
 'CHI': 83239.87033864608,
 'PHO': 83802.87248736476,
 'OKC': 83096.43457608724,
 'POR': 82406.91899227805,
 'GSW': 79614.24592776841,
 'SAC': 83005.47415355571,
 'TOR': 81758.42666238431,
 'ATL': 84402.24616391904,
 'UTA': 82715.12920813449,
 'CLE': 79765.63887501256,
 'LAC': 77766.15556154992}

In [79]:
after_elos = {}
for i, last_game in elo_data[elo_data.year_id == 2015][elo_data.seasongame == 82].iterrows():
    after_elos[last_game.team_id] = last_game.elo_n
sorted(after_elos.items())

[('ATL', 1594.2922),
 ('BOS', 1551.8619),
 ('BRK', 1467.5382),
 ('CHI', 1583.3149),
 ('CHO', 1400.7305),
 ('CLE', 1630.6193),
 ('DAL', 1564.1244),
 ('DEN', 1422.3618),
 ('DET', 1461.0784),
 ('GSW', 1772.3469),
 ('HOU', 1639.3665),
 ('IND', 1504.8713),
 ('LAC', 1705.4121),
 ('LAL', 1283.2395),
 ('MEM', 1587.3375),
 ('MIA', 1455.8601),
 ('MIL', 1472.0676),
 ('MIN', 1263.958),
 ('NOP', 1539.7662),
 ('NYK', 1255.6846),
 ('OKC', 1583.2285),
 ('ORL', 1311.8915),
 ('PHI', 1276.1891),
 ('PHO', 1466.8416),
 ('POR', 1579.5515),
 ('SAC', 1418.2802),
 ('SAS', 1732.7263),
 ('TOR', 1547.5518),
 ('UTA', 1555.2854),
 ('WAS', 1508.7924)]